In [1]:
from transformers import BertForMaskedLM, AutoConfig,AutoTokenizer
lm_config = AutoConfig.from_pretrained('google-bert/bert-base-cased')
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-cased')
lm_model = BertForMaskedLM(lm_config)

lm_model.get_input_embeddings().weight.shape[1]

c:\Users\inova\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\inova\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\inova\.cache\huggingface\hub\models--google-bert--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as

768

In [2]:
from engine.tasks.pretraining import MlmHeadlessPretraining
config = 'configs\mlm_headless_baby_lm.json'

In [3]:
lm_model.get_input_embeddings()


Embedding(28996, 768, padding_idx=0)

In [4]:

task = MlmHeadlessPretraining(
    tokenizer, lm_model, config = config
)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [5]:
task.loss

<bound method MlmHeadlessPretraining.loss of MlmHeadlessPretraining(
  (mlm_model): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (de

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
import numpy as np
import psutil
import functools
import operator
import argparse
import torch
import json


c:\Users\Ino\anaconda3\envs\gpu_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds_name = "cambridge-climb/BabyLM"
ds_config = "strict_small"

load_dataset(ds_name,ds_config,trust_remote_code = True)

DatasetDict({
    train: Dataset({
        features: ['text', 'tagged_text', 'filename'],
        num_rows: 335858
    })
    validation: Dataset({
        features: ['text', 'tagged_text', 'filename'],
        num_rows: 322071
    })
    test: Dataset({
        features: ['text', 'tagged_text', 'filename'],
        num_rows: 342095
    })
})

In [3]:

config = json.load(open('configs\preprocess_baby_lm.json', 'rb'))
ds_name = config["dataset_name"]
ds_config = config["dataset_config"]
hf_tokenizer = config["hf_tokenizer"]
max_seq_len = config["max_seq_len"]
output = config["output"]

NUM_CPU = psutil.cpu_count()
print(f"Using {NUM_CPU} CPUs...")

tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer)






Using 12 CPUs...


In [4]:
def tokenize_and_pack(batch, max_seq_len=max_seq_len):
    tokenized_batch = tokenizer(batch["text"]).input_ids
    tokenized_batch_flat = functools.reduce(operator.iconcat, tokenized_batch, [])
    packed_batch = np.reshape(tokenized_batch_flat[:-(len(tokenized_batch_flat)%max_seq_len)], (-1, max_seq_len))
    return packed_batch.tolist()

In [5]:


print("Loading dataset...")
ds = load_dataset(ds_name,ds_config,trust_remote_code = True)



print("Packing dataset...")
ds = ds.map(lambda x: {"packed":tokenize_and_pack(x)}, remove_columns=ds['train'].column_names, batched=True, batch_size=100000, num_proc=NUM_CPU)

ds = ds.shuffle()

print("Saving dataset...")
ds.save_to_disk(output, num_proc=NUM_CPU)

Loading dataset...
Packing dataset...


Map (num_proc=12):   0%|          | 0/335858 [00:06<?, ? examples/s]


NameError: name 'tokenize_and_pack' is not defined